# TITLE

In [42]:
## Where did you download the files? How did you ingeste the data?

In [43]:
import pandas as pd
from ast import literal_eval
pd.options.display.max_columns = None

agg_df = pd.DataFrame()
path = "./"
for  file in os.listdir(path):
    if file.split(".")[-1] == "csv" and file not in 'accidentalidad.csv':
        df_tmp = pd.read_csv(os.path.join(path, file))
        df_tmp.columns = [c.lower() for c in df_tmp.columns]
        df_tmp['file'] = file
        agg_df = pd.concat([agg_df, df_tmp])

In [44]:
agg_df.columns

Index(['x', 'y', 'objectid', 'radicado', 'fecha', 'hora', 'dia', 'periodo',
       'clase', 'direccion', 'direccion_enc', 'cbml', 'tipo_geocod',
       'gravedad', 'barrio', 'comuna', 'diseno', 'dia_nombre', 'mes',
       'mes_nombre', 'x_magnamed', 'y_magnamed', 'longitud', 'latitud', 'file',
       'shape'],
      dtype='object')

## How did you merge the two DataFrames (aggregated data from 2014-2019 and accidentalidad.csv)

In [45]:
df =  pd.read_csv('./accidentalidad.csv', skiprows=1,header=None,sep=';')
df.loc[0:14]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
0,2019-06-04T20:44:15.587Z,NaN,A000010886,No Ubicada,0,Atropello,"[-75.7037762763, 6.22141524356]",3.0,2014.0,NaN,[u'incidentes'],Tramo 9 Kilometro 999,Operador2,C:/ELK/logstash-6.5.1/data/Medata/Incidentes/I...,Sin Inf,Tramo de via,NaN,Con heridos,2014-03-12T10:30:00.000Z,1180000.00,820000.00,1.0,8626.0,1432623
1,2019-06-04T20:44:15.588Z,809,A000010843,Malla vial,CL 052 A 032 000 00000,Caída de Ocupante,"[-75.5534544709, 6.24349597268]",3.0,2014.0,Villa Hermosa,[u'incidentes'],CL 52 A CR 32,Operador2,C:/ELK/logstash-6.5.1/data/Medata/Incidentes/I...,08,Tramo de via,Sucre,Con heridos,2014-03-12T10:00:00.000Z,1182393.81,836648.68,1.0,8628.0,1432625
2,2019-06-04T20:44:15.588Z,NaN,A000010821,No Ubicada,0,Caída de Ocupante,"[-75.7037762763, 6.22141524356]",3.0,2014.0,NaN,[u'incidentes'],CR 62 CL 84,Operador2,C:/ELK/logstash-6.5.1/data/Medata/Incidentes/I...,Sin Inf,Tramo de via,NaN,Con heridos,2014-03-13T00:20:00.000Z,1180000.00,820000.00,1.0,8630.0,1432627
3,2019-06-04T20:44:15.588Z,514,A000010849,Malla vial,CR 073 B 091 000 00000,Atropello,"[-75.5792346017, 6.28690605219]",3.0,2014.0,Castilla,[u'incidentes'],CR 73 B CL 91,Operador2,C:/ELK/logstash-6.5.1/data/Medata/Incidentes/I...,05,Tramo de via,Alfonso López,Con heridos,2014-03-12T23:00:00.000Z,1187204.11,833808.55,1.0,8632.0,1432630
4,2019-06-04T20:44:15.589Z,1621,A000010855,Malla vial,CR 053 030 A 000 00000,Otro,"[-75.5778859663, 6.23258248117]",3.0,2014.0,Belén,[u'incidentes'],CR 53 CL 30 A,Operador2,C:/ELK/logstash-6.5.1/data/Medata/Incidentes/I...,16,Tramo de via,Cerro Nutibara,Con heridos,2014-03-13T04:50:00.000Z,1181194.15,833940.67,1.0,8639.0,1432637
5,2019-06-04T20:44:15.589Z,6098,A000010826,Malla vial,CR 104 064 000 00000,Caída de Ocupante,"[-75.6151738325, 6.281156378]",3.0,2014.0,Corregimiento de San Cristóbal,[u'incidentes'],CR 104 CL 64,Operador2,C:/ELK/logstash-6.5.1/data/Medata/Incidentes/I...,60,Tramo de via,Área de Expansión Pajarito,Con heridos,2014-03-13T01:00:00.000Z,1186579.60,829828.47,1.0,8642.0,1432640
6,2019-06-04T20:44:15.589Z,1101,A000010917,Malla vial,CR 065 051 000 00000,Choque,"[-75.58085586, 6.25657541199]",3.0,2014.0,Laureles Estadio,[u'incidentes'],CR 65 CL 51,Operador2,C:/ELK/logstash-6.5.1/data/Medata/Incidentes/I...,11,Tramo de via,Carlos E. Restrepo,Con heridos,2014-03-12T15:00:00.000Z,1183849.30,833619.47,1.0,8649.0,1432721
7,2019-06-04T20:44:15.589Z,1318,A000010841,Malla vial,CL 044 105 000 00000,Choque,"[-75.6200332483, 6.25681851817]",3.0,2014.0,San Javier,[u'incidentes'],CL 44 CR 105,Operador2,C:/ELK/logstash-6.5.1/data/Medata/Incidentes/I...,13,Tramo de via,Antonio Nariño,Con heridos,2014-03-12T23:00:00.000Z,1183888.76,829282.64,1.0,8650.0,1432722
8,2019-06-04T20:44:15.590Z,307,A000010916,Malla vial aproximada: CR 45-67,CR 046 067 000 00000,Caída de Ocupante,"[-75.5555813857, 6.2631762147]",3.0,2014.0,Manrique,[u'incidentes'],CR 46 CL 67,Operador2,C:/ELK/logstash-6.5.1/data/Medata/Incidentes/I...,03,Tramo de via,Manrique Central No. 2,Con heridos,2014-03-12T17:00:00.000Z,1184571.57,836419.35,1.0,8659.0,1432732
9,2019-06-04T20:44:15.590Z,1013,A000010992,Malla vial,CR 045 048 000 00000,Caída de Ocupante,"[-75.568553277, 6.24524859423]",3.0,2014.0,La Candelaria,[u'incidentes'],CR 45 CL 48,Operador2,C:/ELK/logstash-6.5.1/data/Medata/Incidentes/I...,10,Tramo de via,Barrio Colón,Con heridos,2014-03-12T16:30:00.000Z,1182592.40,834977.79,1.0,8663.0,1432736


In [46]:
df.columns

Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
            17, 18, 19, 20, 21, 22, 23],
           dtype='int64')

In [53]:
# dic = literal_eval(df[df[0]=="1.0"][4].values[0])
# print(dic)
# df.columns = [col.lower() for col in dic.keys()]
# print(df.columns)

cols = ["timestamp", "cbml", "expediente", "tipo_geocode", "direccion_enc", "clase", "x_long_y_lat", "mes", "periodo", "comuna", "incidente", "direccion", "operador", "path", "num_comuna", "diseno", "barrio", "gravedad", "fecha", "y_magnamed", "x_magnet", "version", "id", "radicado"]
df.columns = cols
df = df[df.timestamp != '1.0']
df[df.timestamp == '1.0']


,timestamp,cbml,expediente,tipo_geocode,direccion_enc,clase,x_long_y_lat,mes,periodo,comuna,incidente,direccion,operador,path,num_comuna,diseno,barrio,gravedad,fecha,y_magnamed,x_magnet,version,id,radicado


In [54]:
agg_df = agg_df[~agg_df.radicado.isna()]
df=df[~df.radicado.isna()]
dtype = dict(radicado=str)
agg_df.radicado = agg_df.radicado.astype(dtype)
df.radicado = df.radicado.astype(dtype)
agg_df.radicado = agg_df.radicado.str.strip()
agg_df.radicado = agg_df.radicado.str.replace("(\d+)(.0)*","\\1") #.dtype #.sort_values(by="radicado", ascending=False)
agg_df.radicado = agg_df.radicado.str.replace("^(0+)(\d+)", "\\2")
df.radicado = df.radicado.str.strip()
df.radicado = df.radicado.str.replace("(\d+)(.0)*","\\1")

In [55]:
df.shape

(277752, 24)